In [39]:
! pip install -U hopsworks --quiet

In [40]:
import pandas as pd
import hopsworks
import joblib

project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3342
Connected. Call `.close()` to terminate connection gracefully.


In [41]:
! pip install dataframe-image --quiet
from datetime import datetime
now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

## Load the Model from Model Registry

In [42]:
mr = project.get_model_registry()
model = mr.get_model("car_prices", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/car_prices_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.

In [43]:
try:
  feature_view = fs.get_feature_view(name="car_prices_inf", version=1)
except:
  car_fg  = fs.get_feature_group(name="car_prices_inference", version=1)
  query = car_fg .select_all()
  feature_view = fs.create_feature_view(name="car_prices_inf",
                                        version=1,
                                        description="Read from car price dataset for inference",
                                        labels=["selling_price"],
                                        query=query)

In [44]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

In [45]:
import datetime
from PIL import Image

batch_data = feature_view.get_batch_data()

td_car = batch_data.sample()

td_car_pred = td_car.drop('full_name', axis = 1)

predictions = model.predict(td_car_pred)

In [46]:
# td_car

In [47]:
# monitor_fg = fs.get_or_create_feature_group(name="car_predictions",
#                                   version=1,
#                                   primary_key=["datetime"],
#                                   description="Car Price Prediction/Outcome Monitoring"
#                                  )

In [48]:
monitor_fg = fs.get_or_create_feature_group(name="car_predictions",
                                  version=2,
                                  primary_key=["datetime"],
                                  description="Car Price Prediction/Outcome Monitoring"
                                 )
data = {
    'Predicted_price_in_Lakhs': predictions[-1],
    'Car_name': td_car.iloc[0][0],
    'Age_of_the_car_in_Years' : td_car.iloc[0][5],
    'Max_power_in_BHP': td_car.iloc[0][4],
    'Kilometers_driven' : td_car.iloc[0][1],
    'datetime': [now],
}
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3342/jobs/named/car_predictions_2_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fa3b5e7e310>, None)

In [49]:
# from datetime import datetime
# now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")

# data = {
#     'prediction': predictions[-1],
#     'label': td_car.iloc[0][0],
#     'datetime': [now],
# }
# monitor_df = pd.DataFrame(data)
# monitor_fg.insert(monitor_df)

In [50]:
history_df = monitor_fg.read()
history_df.rename(columns={'predicted_price_in_lakhs': 'Predicted Price(in Lakhs)',
                           'car_name': 'Name', 'age_of_the_car_in_years': 'Age(in years)',
                           'max_power_in_bhp': 'Max Power(in BHP)', 'kilometers_driven': 'KM Driven'}, inplace=True)
history_df

,Predicted Price(in Lakhs),Name,Age(in years),Max Power(in BHP),KM Driven,datetime
0,16.2724,Toyota Fortuner 4x4 AT,7.0,168.5,290000,"11/29/2022, 08:48:11"
1,16.1150,Toyota Innova Crysta 2016-2020 2.8 GX AT BSIV,7.0,171.5,60100,"11/29/2022, 09:14:16"


In [51]:
import dataframe_image as dfi

df_recent = history_df.tail(10)
 
dfi.export(df_recent, 'assets/df_recent.png', table_conversion = 'matplotlib')

In [52]:
history_df.to_csv('assets/output.csv')